# Consuming the Linear Model

Having a model deployed as a web API is a perfect start, but we shouldn't stop there. We can use interactive notebooks to show our customers (internal and external) how to access the model and explain our thinking.
Make sure to execute the following commands in the root directory of this repository first to serve the models and start the API.

```bash
dotnet serve -d ./models/ -p 8081
```

```bash
dotnet run -p ./src/LoadForecasting.App/LoadForecasting.App.fsproj
```

In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

As usual we load the data for evaluatin purposes and give it a quick glance.

In [ ]:
open Deedle
open Plotly.NET
open System
open System.Net.Http
open System.Net.Http.Json
open System.Text.Json
open System.Text.Json.Serialization

let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

Speaking about sanity checks: it makes sense to smoke test your application before a demo. Besides this very easy endpoint we could configure arbitrarily complex health probes using common ASP.NET Core APIs.

In [ ]:
let baseUrl = "http://localhost:5000"

let client = new HttpClient()
client.BaseAddress <- Uri(baseUrl)

client.GetAsync("/")
|> Async.AwaitTask
|> Async.RunSynchronously

The design behind the current API tries to make calling it as simple as possible. Just provide a start date and a horizon (how many hours from this date do you want to forecast into the future) and you should get your results back.

In [ ]:
type LinearForecastResponse =
    { LoadForecast: float }

let from = DateTime.Parse("2019-03-10T00:00:00")
let horizon = 24 * 10

let response =
    client.PostAsJsonAsync("/linear-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues =
    response.Content.ReadFromJsonAsync<LinearForecastResponse array>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> Array.map (fun lf -> lf.LoadForecast)

forecastedValues
|> Array.take 10

There are some analysists who could tell you a whole lot about the forecast (and its validity) just from looking at the raw numbers. Most people favor visualizations, though. We can pretty much recycle the same approach we used earlier in our own evaluation.

In [ ]:
let keys, actualValues =
    data?Value
    |> Series.observations
    |> Seq.skipWhile (fun (k, _) -> k <> from)
    |> Seq.take horizon
    |> Seq.unzip

let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine